In [0]:
%run "../includes/configurations"

In [0]:
print({presentation_folder_path})

In [0]:
from pyspark.sql.functions import col

In [0]:
race_results_df=spark.read.parquet("/mnt/formula1dljit/presentation/race_results")
                                   

In [0]:
display(race_results_df)

In [0]:
from pyspark.sql.functions import sum,when,count

In [0]:
drivers_standing_df=race_results_df.groupBy("race_year","driver_name","driver_nationality","team").\
agg(sum(col("points")).alias("total_points"),
    count(when(col("positionOrder")==1,1).alias("wins")).alias("wins")
    ).orderBy(col("race_year").desc(),col("total_points").desc())

In [0]:
display(drivers_standing_df)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

In [0]:
drive_rann_spec=Window.partitionBy("race_year").orderBy(col("total_points").desc(),col("wins").desc())

final_df=drivers_standing_df.withColumn("rank",rank().over(drive_rann_spec))

In [0]:
#final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standing")

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standing")

In [0]:
spark.read.parquet(f"{presentation_folder_path}/driver_standing").display()